# Calculating the Water Permeability of a LPS Bilayer by Counting Crossings

I am using a few Python packages here.

Numpy and pandas are available through anaconda's default channels

```
conda install numpy pandas
```

mdtraj and openmm are available on the omnia channel:
```
conda install -c omnia openmm mdtraj
```

rflow is my package. I have not uploaded it to conda yet, so you will have to clone it from gitlab, cd into the directory, and run
```
python setup.py install
```

In [1]:
import numpy as np
import pandas as pd
from simtk.openmm.app import CharmmPsfFile
from rflow import TrajectoryIterator, PermeationEventCounter, normalize
import mdtraj as md 

Creating an iterator over the trajectory files. 

This is a bit circumvential if you have only one trajectory file, 
but it is designed so that you can run the analysis on subsequent trajectories, (here they would be named *dyn1.dcd*, ..., *dyn100.dcd* )).

In [2]:
trajectories = TrajectoryIterator(
    first_sequence=1, last_sequence=1,
    filename_template="dyn{}.dcd",
    topology_file="Ca_LPS.psf"
)

(I renamed your trajectory into *dyn1.dcd*.)

## Get a Reasonable Dividing Surface

The dividing surface defines the border between the hydrophobic core of the bilayer and the water phase.
In our applications, a good choice for the dividing surface was the phosphate plane. 
For water permeation, the number of crossings is not very sensitive wrt. the exact placement of the dividing surface. 

In [3]:
water_oxygens = trajectories.topology.select("water and mass > 2")
center_lipids = trajectories.topology.select("resname LIPA")
phosphates = trajectories.topology.select("name PA")

In [4]:
traj = next(iter(trajectories))

This command just returns the trajectory from the trajectory iterator. If you work on multiple files, just use

```
for traj in trajectories:
    ... do stuff ...
```

Now, lets get the z coordinates of the phosphate relative to the box height, i.e. all normalized coordinates are between 0 and 1, the center of the bilayer being at 0.5.

In [5]:
normalized_z = normalize(traj, 2, center_lipids, phosphates)

In [6]:
normalized_z.mean()

0.4989579041534057

As expected, the phosphates are nicely centered around the normalized center 0.5.

In [7]:
print(np.abs(normalized_z - 0.5).mean().round(3))

0.14


The phosphate plane is at  $z_\mathrm{normalized} = 0.5 \pm 0.14.$
I played around with this dividing surface a little bit and noticed that the one water that permeating molecule started off in between the phosphate planes. As a result, no permeation event was detected. I had to shift the diving surface slightly inward to $z_\mathrm{normalized} = 0.5 \pm 0.1.$

## Counting Crossings

In addition to the dividing surface, we have to specify the *center region*. 
If it is chosen too small, you miss crossings -- the PermeationEventCounter will let you know through a warning.
If it is chosen too big, you get a lot of *fake entries and rebounds*. We will not have to worry that much, because
we are mainly interested in crossings. I found that 20% of the dividing surface is a good value for your system.

In [8]:
pcount = PermeationEventCounter(center_threshold=0.02, 
                                dividing_surface=0.1, 
                                solute_ids=water_oxygens, 
                                membrane=center_lipids)

for traj in trajectories:
    pcount(traj)

The events are stored in a dictionary `pcount.events`. I convert it to a pandas.DataFrame for nicer printing: 

In [9]:
events = pd.DataFrame(pcount.events)
events

,atom,crossing_time_nframes,entry_time_nframes,exit_time_nframes,frame,from_water,rebound_time_nframes,type
0,36495,NaN,30.0,NaN,343,1,NaN,entry
1,36495,NaN,NaN,178.0,521,1,208.0,rebound
2,34425,NaN,210.0,NaN,610,1,NaN,entry
3,38127,NaN,105.0,NaN,624,1,NaN,entry
4,34425,NaN,NaN,15.0,625,1,225.0,rebound
5,38127,146.0,NaN,8.0,665,1,NaN,crossing


Finally, we save this table to a file and print out the number of crossings:

In [15]:
events.to_csv("permeation_events.csv", na_rep="NaN")
print("#Crossings:", (events.type=="crossing").sum())

#Crossings: 1


Relating the number of crossings to the permeability is easy. Let's do that once we have observed more events.